In [1]:
import os, shutil
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from time import time
import numpy as np
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
import random
import metrics

from keras.models import Model
from keras import backend as K
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model
import numpy as np

/Users/orobix/opt/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/orobix/opt/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/orobix/opt/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/orobix/opt/anaconda3/envs/tesi

## Data aquisition

In [2]:
CT = '../Data/CT/'
MRI = '../Data/MRI/'
PET = '../Data/PET/'

myPaths = [CT, MRI, PET]
myDict = {CT:[], MRI:[], PET:[]}

#images file names dictionary
for path in myPaths:
    myDict[path] = [f for f in listdir(path) if isfile(join(path, f))]

### Split data in train, validation and test dataset

In [14]:
#70:20:10 
train_size=0.7
val_size=0.2

#split train test
CT_train, CT_test = train_test_split(myDict[myPaths[0]], train_size=0.70)
MRI_train, MRI_test = train_test_split(myDict[myPaths[1]], train_size=0.70)
PET_train, PET_test = train_test_split(myDict[myPaths[2]], train_size=0.70)

#split train validation
CT_val, CT_test = train_test_split(CT_test, train_size=(val_size)/(1-train_size))
MRI_val, MRI_test = train_test_split(MRI_test, train_size=(val_size)/(1-train_size))
PET_val, PET_test = train_test_split(PET_test, train_size=(val_size)/(1-train_size))

#create list of file names
train_file_names = CT_train + MRI_train + PET_train
val_file_names = CT_val + MRI_val + PET_val
test_file_names = CT_test + MRI_test + PET_test

#print dimensions datasets
lenTot = len(train_file_names)+len(val_file_names)+len(test_file_names)
percTrain = (len(train_file_names)/lenTot)*100
percVal = (len(val_file_names)/lenTot)*100
percTest = (len(test_file_names)/lenTot)*100
print('Number of train images:', len(train_file_names), '= %.0f' % percTrain,'%')
print('Number of validation images:', len(val_file_names), '= %.0f' % percVal,'%')
print('Number of test images:', len(test_file_names), '= %.0f' % percTest,'%')

Number of train images: 744 = 70 %
Number of validation images: 213 = 20 %
Number of test images: 108 = 10 %


### Make data/ and copy images in train/, val/ and test/

In [17]:
print(os.getcwd())

#delete existing data/
data_dir = 'data/'
if os.path.exists(data_dir):
    print('Deleting', data_dir)
    try:
        shutil.rmtree(data_dir)
    except:
        raise
print('Delete complete')

/Users/orobix/Documents/Thesis/Source/Code
Delete complete


In [18]:
#make data/
print('Creating', data_dir, 'here:', os.getcwd())

directories = ['data/train/', 
               'data/val/', 
               'data/test/']

#make
for directory in directories:
        os.makedirs(directory)

Creating data/ here: /Users/orobix/Documents/Thesis/Source/Code


In [21]:
#copy images in relative dir
_file_names = [train_file_names, val_file_names, test_file_names] 

def copyImages(srcs, _file_names, directory):
    for source in srcs:
        for f in _file_names:
            try:
                shutil.copy(source+f, directory)
            except OSError:
                pass

for name,directory in zip(_file_names, directories):
    copyImages(myPaths, name, directory)